In [4]:
import dgl
import numpy as np
import torch
import scipy as sp
from dgl.heterograph import DGLBlock

In [6]:
import importlib
import packages.data_management.pkl_io as pkl_io
# import packages.transformer.data as transformer_data 
# importlib.reload(pkl_io)
# importlib.reload(transformer_data)
# import packages.utils.sp_utils as sp_utils
# importlib.reload(sp_utils)


In [8]:
# Local imports 
from packages.data_management.pkl_io import save_pkl, load_pkl_from_path, load_npy, save_npy
from packages.utils.sp_utils import select_submatrix, convert_scipy_sparse_to_torch
from packages.transformer.data import retrieve_features_for_minibatch, retrieve_labels_for_minibatch, TransformerGraphBundleInput
from packages.transformer.encoder_decoder import make_model

In [9]:
def load_reddit_adj(path="data/reddit_cpr", fname="reddit_adj_coo.npz"):
    adj = sp.sparse.load_npz(f'{path}/{fname}')
    return adj

def load_reddit_feats(path="data/reddit_cpr", fname="reddit_feats.npy"):
    with open(f"{path}/{fname}", "rb") as f:
        feats = np.load(f)
        return feats

def load_reddit_labels(path="data/reddit_cpr", fname="reddit_labels.npy"):
    with open(f"{path}/{fname}", "rb") as f:
        labels = np.load(f)
        return labels 

def load_reddit_masks(path="data/reddit_cpr", fname="reddit_masks.npy"):
    with open(f"{path}/{fname}", "rb") as f:
        masks = np.load(f)
        return masks 

In [10]:
adj_sparse = load_reddit_adj()
print(adj_sparse.shape)

(232965, 232965)


In [11]:
feats = load_reddit_feats()
print(feats.shape)

In [5]:
masks = load_reddit_masks()
print(masks.shape)

(3, 232965)


In [63]:
labels = load_reddit_labels()
print(labels.shape)

(232965,)


In [6]:
all_ids = np.arange(masks.shape[1])
train_ids = all_ids[masks[0,:]]

In [7]:
graph = dgl.graph((adj_sparse.row, adj_sparse.col))


In [8]:
sampler = dgl.dataloading.MultiLayerNeighborSampler([5, 5])
dataloader = dgl.dataloading.DataLoader(
    graph, train_ids, sampler,
    batch_size=32,
    shuffle=True,
    drop_last=False,
    num_workers=0)

In [82]:
from torch import square

def convert_mfg_to_sg_adj(mfg: DGLBlock, square_shape: int):
    sparse_adj = mfg.adj()
    square_adj = torch.sparse_coo_tensor(sparse_adj._indices(), sparse_adj._values(), size=(square_shape, square_shape)) 
    return square_adj.to_dense()

def construct_batch(target_nodes, subgraph_nodes, mfgs, features, labels, device):
    first_layer_mfg = mfgs[0]
    second_layer_mfg = mfgs[1]

    first_layer_adj_submatrix = convert_mfg_to_sg_adj(first_layer_mfg, subgraph_nodes.shape[0]) + torch.eye(subgraph_nodes.shape[0], device=device) 
    second_layer_adj_submatrix = convert_mfg_to_sg_adj(second_layer_mfg, subgraph_nodes.shape[0]) + torch.eye(subgraph_nodes.shape[0], device=device) 
    
    minibatch_adjacencies = torch.stack((first_layer_adj_submatrix, second_layer_adj_submatrix))
    all_minibatch_feats = features[subgraph_nodes, :]

    all_minibatch_feats = all_minibatch_feats.unsqueeze(0)
    minibatch_adjacencies = minibatch_adjacencies.unsqueeze(0)
    minibatch_labels = labels[target_nodes].unsqueeze(0)
    output_node_inds = target_nodes.unsqueeze(0)

    minibatch = TransformerGraphBundleInput(all_minibatch_feats, minibatch_labels, minibatch_adjacencies, output_node_inds, device)
    return minibatch

In [9]:
dataloader_iter = iter(dataloader)
input_nodes, output_nodes, mfgs = next(dataloader_iter) # input nodes gives us the requisite features. The mfgs gives us the requisite attention mask

In [70]:
unique_labels = torch.Tensor(labels).unique()

In [83]:
mfgs = load_pkl_from_path("data/reddit_cpr/mfg")
all_sg_nodes = mfgs[0].srcdata[dgl.NID]

all_tgt_nodes = mfgs[1].dstdata[dgl.NID]
minibatch = construct_batch(all_tgt_nodes, all_sg_nodes, mfgs, torch.Tensor(feats), torch.Tensor(labels), 'cpu')


TypeError: __init__() missing 1 required positional argument: 'device'

In [74]:
model = make_model(feats.shape[1], len(unique_labels), N=2) # TODO: do i need a +1 here?? DOn't think so.

/home/fsamir/gnn/packages/transformer/encoder_decoder.py:142: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


In [79]:
model.forward(minibatch.src_feats, minibatch.src_mask, minibatch.train_inds)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_mm)

In [32]:
adj_sparse = convert_scipy_sparse_to_torch(adj_sparse, 'cpu')

In [33]:
minibatch = construct_batch(output_nodes, input_nodes, mfgs, adj_sparse, feats, 'cpu')

: 

: 

In [3]:
def get_max_nodes_in_batch(batch_size, fanout):
    print(batch_size + batch_size * fanout + (batch_size + batch_size * fanout) * fanout)

get_max_nodes_in_batch(64, 5)
get_max_nodes_in_batch(64, 4)
get_max_nodes_in_batch(32, 5)

2304
1600
1152
